In [65]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

class HISHotelsScraper:
    def __init__(self, database_name='hotels.db'):
        self.db_name = database_name
        self.create_table()

    def create_table(self):
        """データベースにテーブルを作成する"""
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        c.execute('''
            CREATE TABLE IF NOT EXISTS hotels (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                price TEXT,
                area TEXT
            )
        ''')
        conn.commit()
        conn.close()

    def scrape_hotels(self, url, area):
        """指定されたURLからホテル名と価格をスクレイピングし、データベースに保存する"""
        response = requests.get(url)
        response.raise_for_status()  # ステータスコードが200以外の場合例外を発生させる
        soup = BeautifulSoup(response.content, 'html.parser')

        # ホテル情報を含む要素を探す
        search_items = soup.find_all('div', class_='item-wrap__main')

        # データベースにホテル名と金額を保存する
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        for item in search_items:
            name_tag = item.find('span', class_='item-wrap__title-ja')
            price_tag = item.find('span', class_='price--body')
            if name_tag and price_tag:
                hotel_name = name_tag.get_text(strip=True)
                hotel_price = price_tag.get_text(strip=True)
                c.execute('INSERT INTO hotels (name, price, area) VALUES (?, ?, ?)', (hotel_name, hotel_price, area))
                print(f'Inserting hotel name into DB: {hotel_name}, Price: {hotel_price}, Area: {area}')  # デバッグ用出力

        conn.commit()
        conn.close()

    def get_hotels(self):
        """データベースからホテル名と価格を取得する"""
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        c.execute('SELECT name, price, area FROM hotels')
        hotels = c.fetchall()  # 取得したデータをすべてfetchする
        conn.close()
        return hotels

    def clear_hotels(self):
        """テーブル内の既存のデータをクリア"""
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        c.execute('DELETE FROM hotels')
        conn.commit()
        conn.close()



In [66]:
def main():
    urls_and_areas = [
        ('https://hotel.his-j.com/search/?isAutoSelect=true&checkInDate=20250201&checkOutDate=20250202&prefecture=40&area=259&room=1&room1Adult=1', '福岡市'),
        ('https://hotel.his-j.com/search/?t=KoDl3QI0sklQrCJb&stayPrefecture=40&stayArea=259&stayDistrict=&hotelCode=&checkInDate=20250201&checkOutDate=20250202&room1Adult=1&sortHotelKey=recommended&prefecture=40&area=258&district=&minRate=0&maxRate=', '北九州'),
        ('https://hotel.his-j.com/search/?t=GRcuxqez8wDYJAMF&stayPrefecture=40&stayArea=258&stayDistrict=&hotelCode=&checkInDate=20250201&checkOutDate=20250202&room1Adult=1&sortHotelKey=recommended&prefecture=40&area=260&district=&minRate=0&maxRate=', '太宰府'),
        ('https://hotel.his-j.com/search/?t=2LB3Eg02TNiWeHvz&stayPrefecture=40&stayArea=260&stayDistrict=&hotelCode=&checkInDate=20250201&checkOutDate=20250202&room1Adult=1&prefecture=40&area=261&district=&minRate=0&maxRate=', '宗像'),
        ('https://hotel.his-j.com/search/?t=22CNCIODFxBUufqi&stayPrefecture=40&stayArea=261&stayDistrict=&hotelCode=&checkInDate=20250201&checkOutDate=20250202&room1Adult=1&sortHotelKey=recommended&prefecture=40&area=263&district=&minRate=0&maxRate=', '久留米'),
        ('https://hotel.his-j.com/search/?t=cVvc906fcadc7G6m&stayPrefecture=40&stayArea=263&stayDistrict=&hotelCode=&checkInDate=20250201&checkOutDate=20250202&room1Adult=1&sortHotelKey=recommended&prefecture=40&area=257&district=&minRate=0&maxRate=', '柳川')
    ]

    scraper = HISHotelsScraper()

    # データベース内の既存のデータをクリア
    scraper.clear_hotels()

    # 各URLを順に処理
    for url, area in urls_and_areas:
        # サーバ負荷に配慮してリクエスト間隔を設ける
        time.sleep(2)
        scraper.scrape_hotels(url, area)

    # データベースに保存されたホテル名と金額を取得して表示
    hotels = scraper.get_hotels()
    for hotel in hotels:
        print(f'Hotel from DB: Name: {hotel[0]}, Price: {hotel[1]}, Area: {hotel[2]}')

if __name__ == "__main__":
    main()

Inserting hotel name into DB: ホテルモントレ福岡, Price: 41,090, Area: 福岡市
Inserting hotel name into DB: ホテルモントレ ラ・スール福岡, Price: 27,250, Area: 福岡市
Inserting hotel name into DB: lyf Tenjin Fukuoka（ライフ天神福岡）, Price: 34,380, Area: 福岡市
Inserting hotel name into DB: 変なホテル福岡博多, Price: 23,730, Area: 福岡市
Inserting hotel name into DB: ヒルトン福岡シーホーク, Price: 54,820, Area: 福岡市
Inserting hotel name into DB: ホテルウィングインターナショナルセレクト博多駅前, Price: 32,900, Area: 福岡市
Inserting hotel name into DB: ANAクラウンプラザホテル福岡, Price: 42,500, Area: 福岡市
Inserting hotel name into DB: キャナルシティ福岡ワシントンホテル, Price: 17,820, Area: 福岡市
Inserting hotel name into DB: 東洋ホテル, Price: 27,850, Area: 福岡市
Inserting hotel name into DB: スカイハートホテル博多, Price: 17,800, Area: 福岡市
Inserting hotel name into DB: ザ・ライブリー福岡博多（THE LIVELY　福岡博多）, Price: 30,500, Area: 福岡市
Inserting hotel name into DB: ホテルトリフィート博多祇園, Price: 54,080, Area: 福岡市
Inserting hotel name into DB: the b 博多, Price: 33,000, Area: 福岡市
Inserting hotel name into DB: 東急ステイ福岡天神, Price: 20,000, Area: 福岡市
I

In [68]:
import sqlite3

def calculate_average_prices_by_area(database_name='hotels.db'):
    conn = sqlite3.connect(database_name)
    c = conn.cursor()
    
    query = '''
        SELECT area, AVG(CAST(REPLACE(price, ',', '') AS REAL)) as average_price
        FROM hotels
        GROUP BY area
    '''
    
    c.execute(query)
    results = c.fetchall()
    conn.close()
    
    print("エリアごとの平均料金:")
    for area, avg_price in results:
        print(f"エリア: {area}, 平均料金: {avg_price:.2f}円")

if __name__ == "__main__":
    calculate_average_prices_by_area()

エリアごとの平均料金:
エリア: 久留米, 平均料金: 9400.00円
エリア: 北九州, 平均料金: 17245.93円
エリア: 太宰府, 平均料金: 12700.00円
エリア: 宗像, 平均料金: 17940.00円
エリア: 柳川, 平均料金: 11268.33円
エリア: 福岡市, 平均料金: 35511.11円
